# Vocabulary Challenge

1. Get the book "Los Miserables" by Victor Hugo, from Carlos Slim Foundation. 
   - https://aprende.org/pruebasat?sectionId=6
2. Convert the book into text (CSV)
3. Clean the csv file:
   - Standarize (no upper chars, no whitespace, no punctuation, accents, if possible).
   - Create a vocabulary with the words in the book, following the ideas when discussed how to create a vocabulary in the notes of this slide set.
   - Store the vocabulary in parquet format.
   - Do statistics, including:
     + How many words in book.
     + How many different words in vocabulary.
     + Print the 100 most frequent words.
4. Produce a 2 page report to describe your experience, methods, etc.
5. Write code in a language of your choosing from this set (Go, Julia, Python)
6. Submit by 14/03/2025 @ 16:00 UTC-6 (Mexico City Time), inside the "python/src/student_submissions/Vocabulary" folder.
7. You must create a folder named "lastname_firstname" to put your report.

In [1]:
import re
import pandas as pd
from PyPDF2 import PdfReader
from unicodedata import normalize
from copy import copy
from collections import Counter

Primeramente, definimos las rutas de los archivos y abrimos el archivo pdf.
Con la ayuda de la librería PyPDF convertimos el archivo a texto plano.

In [2]:
path = "/mnt/c/Users/omarm/Downloads/"
name = "Los-miserables"

reader = PdfReader(path+name+'.pdf')
number_of_pages = len(reader.pages)

print(f"El número de páginas es {number_of_pages}")

El número de páginas es 305


In [ ]:
all_text = ''
for page in reader.pages:
    text = page.extract_text()
    all_text+=text+'\n'

print(f"El total de caracteres en el libro es: {len(all_text)}")

El total de caracteres en el libro es: 640742


#### Limpieza del texto

Creamos una función que nos ayudara a limpiar el texto extraído, convirtiendo todo a minúsculas, quitando diacríticos excepto la __ñ__, eliminando números y signos de puntuación, de esta manera logramos un texto plano limpio y que nos permitirá trabajarlo de manera más adecuada.

In [4]:
def clean_text(text: str) -> str:
    text_f = copy(text)
    text_f = text_f.lower()
    # -> NFD y eliminar diacríticos
    text_f = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1",
        normalize("NFD", text_f), 0, re.I)
    text_f = normalize('NFC', text_f)
    text_f = re.sub("\d+", "", text_f)
    text_f = re.sub(r'[^\w0-9()\s]', '', text_f)

    return text_f

La siguiente es una muestra de cómo se ve el texto resultante ya limpiado.

In [5]:
final_text = clean_text(all_text)
print(final_text[12101:12400])

 senador del imperio antiguo miembro del consejo de los quinientos favorable al  brumario y agraciado cerca de la ciudad de digne con una magnifica senaduria escribio al ministro de cultos bigot de preameneu una nota irritada y confidencial de la cual extraemos estas lineas autenticas gastos de car


#### Guardar el vocabulario en formato CSV

Primeramente creamos el vocabulario del libro contando el número de veces que se repite cada palabra y guardándolo en un diccionario, para posteriormente convertirlo a un dataframe y finalmente guardarlo en formato CSV.

In [6]:
vocabulary = Counter(final_text.split())
dict(vocabulary)

{'los': 1352,
 'miserables': 5,
 'i': 9,
 'hugo': 3,
 'victor': 2,
 'novela': 5,
 'se': 1681,
 'reconocen': 3,
 'derechos': 5,
 'morales': 3,
 'de': 5325,
 'obra': 13,
 'dominio': 1,
 'publico': 12,
 'distribucion': 2,
 'gratuita': 3,
 'prohibida': 1,
 'su': 1245,
 'venta': 2,
 'y': 3123,
 'en': 2836,
 'medios': 5,
 'ajenos': 1,
 'a': 2488,
 'la': 3918,
 'fundacion': 2,
 'carlos': 2,
 'slim': 2,
 'lago': 3,
 'zurich': 1,
 'plaza': 17,
 'carso': 1,
 'ii': 10,
 'piso': 14,
 'col': 2,
 'ampliacion': 1,
 'granada': 1,
 'c': 1,
 'p': 3,
 'ciudad': 62,
 'mexico': 2,
 'contactopruebatorg': 1,
 'primera': 55,
 'parte': 60,
 'fantine': 194,
 'libro': 26,
 'primero': 22,
 'un': 1601,
 'justo': 22,
 'el': 3393,
 'señor': 447,
 'myriel': 34,
 'monseñor': 105,
 'charlesfrancoisbienvenu': 1,
 'era': 650,
 'obispo': 286,
 'digne': 57,
 'anciano': 29,
 'cerca': 75,
 'setenta': 3,
 'cinco': 50,
 'años': 136,
 'ocupaba': 8,
 'sede': 3,
 'desde': 99,
 'aunque': 32,
 'este': 261,
 'detalle': 8,
 'no': 149

In [7]:

data = {'word': list(vocabulary.keys()), 'frequency': list(vocabulary.values())}
vocabulary_df = pd.DataFrame.from_dict(data)
vocabulary_df.to_csv(path+name+'.csv', index=False)

#### Guardar el archvo en formato parquet

Guardamos el archivo ayudándonos de la librería pyarrow.

In [8]:
vocabulary_df.to_parquet(path+name+'.parquet', index=False, engine='pyarrow')

In [9]:
# Funcion para mejorar el formato de impresion de las palabras.
def format_codes(codes: list[str], limit: int = 100) -> str:
    """Remove the word _copy and return a string with all codes."""
    short_codes = codes if len(codes) < limit else list(codes)[0:limit]
    msg = ", ".join([c for c in short_codes])
    if len(codes) > limit:
        msg += f" y otros {len(list(codes)[limit:])}."
    return msg

#### Estadística del libro

Finalmente obtenemos algunos datos de relevancia acerca del libro, primeramente, obtuvimos que el total de palabras que contiene es de __109271__, siendo un total de __13125__ palabras distintas las que se utilizan. Se muestran las 100 palabras más comunes y las 100 menos comunes con su respectiva frecuencia, finalmente también se obtiene el valor medio de uso de una palabra en el libro que es de __8.32__.

In [10]:
vocabulary_df = pd.read_parquet(path+name+'.parquet')

print(f"Total de palabras en el libro: {vocabulary_df['frequency'].sum()}")
print(f"Cantidad de palabras distintas en el libro: {len(vocabulary_df)}")

most_com = vocabulary_df.sort_values(by='frequency', ascending=False).head(100)
less_com = vocabulary_df.sort_values(by='frequency', ascending=True).head(100)
mean = vocabulary_df['frequency'].mean()

print(f"Las 100 palabras mas comunes son: {format_codes([f'"{row['word']}" con {int(row.frequency)}' for row in most_com.iloc])}")
print(f"Las 100 palabras menos comunes son: {format_codes([f'"{row['word']}" con {int(row.frequency)}' for row in less_com.iloc])}")

print(f"La frecuencia media de una palabra en el libro es de {mean}")

Total de palabras en el libro: 109271
Cantidad de palabras distintas en el libro: 13135
Las 100 palabras mas comunes son: "de" con 5325, "la" con 3918, "que" con 3818, "el" con 3393, "y" con 3123, "en" con 2836, "a" con 2488, "se" con 1681, "un" con 1601, "no" con 1498, "los" con 1352, "una" con 1316, "su" con 1245, "las" con 935, "por" con 935, "con" con 924, "habia" con 858, "del" con 813, "al" con 756, "es" con 749, "lo" con 719, "le" con 667, "era" con 650, "como" con 571, "mas" con 513, "para" con 504, "señor" con 447, "esta" con 414, "pero" con 372, "hombre" con 363, "si" con 358, "sus" con 344, "todo" con 327, "me" con 326, "sin" con 311, "obispo" con 286, "dijo" con 281, "cuando" con 274, "estaba" con 273, "sobre" con 269, "dos" con 264, "este" con 261, "aquel" con 253, "mi" con 244, "ya" con 229, "hacia" con 219, "esto" con 218, "yo" con 218, "madeleine" con 214, "tenia" con 211, "ha" con 199, "jean" con 199, "fantine" con 194, "valjean" con 192, "aquella" con 190, "hay" con 1